In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd


logits = np.array([2.0, 1.0, 0.1])
labels = ['v1', 'v2', 'v3']

def sample_gumbel(shape, eps=1e-20):
    U = np.random.uniform(0, 1, shape)
    return -np.log(-np.log(U + eps) + eps)

tau = 0.5
samples = 10

logits_data = []

for i in range(samples):
    noise = sample_gumbel(logits.shape)
    for j, label in enumerate(labels):
        logits_data.append({
            'Sample': i,
            'Codeword': label,
            'Noisy Logit': (logits[j] + noise[j]) / tau,
            'Type': 'With Gumbel Noise'
        })
        logits_data.append({
            'Sample': i,
            'Codeword': label,
            'Noisy Logit': logits[j] / tau,
            'Type': 'No Noise'
        })

df = pd.DataFrame(logits_data)

fig = go.Figure()

for codeword in labels:
    df_noise = df[(df['Codeword'] == codeword) & (df['Type'] == 'With Gumbel Noise')]
    df_nonoise = df[(df['Codeword'] == codeword) & (df['Type'] == 'No Noise')]
    
    fig.add_trace(go.Scatter(
        x=df_noise['Sample'],
        y=df_noise['Noisy Logit'],
        mode='lines+markers',
        name=f'{codeword} (w noise)',
        line=dict(dash='solid'),
        opacity=1.0
    ))
    
    fig.add_trace(go.Scatter(
        x=df_nonoise['Sample'],
        y=df_nonoise['Noisy Logit'],
        mode='lines+markers',
        name=f'{codeword} (w/o noise)',
        line=dict(dash='dot'),
        opacity=0.3
    ))

fig.update_layout(
    title='Comparison of Logits With and Without Gumbel Noise',
    xaxis_title='Sample Index',
    yaxis_title='(logit + noise) / tau',
    legend_title='Codewords'
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

x_vals = np.linspace(-5, 5, 200)

argmax_outputs = [1 if x > 0 else 0 for x in x_vals]

tau_0_5 = [np.exp(x / 0.5) / (np.exp(x / 0.5) + 1.0) for x in x_vals]
tau_1_0 = [np.exp(x / 1.0) / (np.exp(x / 1.0) + 1.0) for x in x_vals]

fig = go.Figure()

# argmax 線
fig.add_trace(go.Scatter(
    x=x_vals, y=argmax_outputs,
    mode='lines',
    name='argmax(x, 0)',
    line=dict(dash='dot', color='red')
))

fig.add_trace(go.Scatter(
    x=x_vals, y=tau_0_5,
    mode='lines',
    name='softmax(x, 0), tau=0.5',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=x_vals, y=tau_1_0,
    mode='lines',
    name='softmax(x, 0), tau=1.0',
    line=dict(color='green')
))

fig.update_layout(
    title='Argmax vs Softmax Approximation (tau=0.5 vs 1.0)',
    xaxis_title='Logit x',
    yaxis_title='Probability of Choosing x',
    legend_title='Function Type'
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

num_codewords = 3
logits = np.linspace(1, 2.0, num_codewords)
labels = [f'codeword {i+1}' for i in range(num_codewords)]

np.random.seed(42)
def sample_gumbel(shape, eps=1e-20):
    U = np.random.uniform(0, 1, shape)
    return -np.log(-np.log(U + eps) + eps)

gumbel_noise = sample_gumbel(logits.shape)

taus = [2.0, 1.0, 0.5, 0.1]

softmax_data = []

for tau in taus:
    noisy_logits = (logits + gumbel_noise) / tau
    probs = np.exp(noisy_logits) / np.sum(np.exp(noisy_logits))
    for i in range(num_codewords):
        softmax_data.append({
            'Tau': tau,
            'Codeword': labels[i],
            'Probability': probs[i]
        })

# 建立 dataframe
df = pd.DataFrame(softmax_data)

fig = px.strip(
    df,
    x='Codeword',
    y='Probability',
    color='Tau',
    stripmode='group',
    title='Gumbel-Softmax with different Tau',
    labels={'Probability': 'Selection Probability'},
    category_orders={'Codeword': labels}
)

fig.update_traces(
    jitter=0.4,
    marker=dict(size=10, opacity=0.8),
    selector=dict(type='scatter')
)

fig.update_layout(
    xaxis_title='Codewords',
    yaxis_title='Probability',
    legend_title='tau',
    yaxis=dict(range=[-0.05, 1.05]),
    width=800,
    height=300
)

fig.show()


In [ ]:
import torch
import torch.nn as nn

# batch_size=1, channel=4, time_steps=10
# x = torch.randn(1, 4, 3)

torch.manual_seed(39)
x = torch.tensor([[[1.0, 1.0, 1.0],
                   [0.5, 0.5, 0.5],
                   [1.0, 1.0, 1.0]]])
print(x)

pointwise_conv = nn.Conv1d(in_channels=3, out_channels=6, kernel_size=1)


output = pointwise_conv(x)
print(output)
print(output.shape)


tensor([[[1.0000, 1.0000, 1.0000],
         [0.5000, 0.5000, 0.5000],
         [1.0000, 1.0000, 1.0000]]])
tensor([[[-0.3336, -0.3336, -0.3336],
         [ 0.3315,  0.3315,  0.3315],
         [-0.6426, -0.6426, -0.6426],
         [ 0.4418,  0.4418,  0.4418],
         [ 0.7537,  0.7537,  0.7537],
         [-0.0968, -0.0968, -0.0968]]], grad_fn=<ConvolutionBackward0>)
torch.Size([1, 6, 3])


In [ ]:
import torch
import torch.nn as nn

x = torch.randn(2, 64, 100)
pointwise_conv = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
out = pointwise_conv(x)
print(out.shape)

depthwise_conv = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1, groups=128)
out2 = depthwise_conv(out)
print(out2.shape)

glu_input = torch.randn(2, 128, 100)
glu = nn.GLU(dim=1)
print(glu_input.shape)
out3 = glu(glu_input)
print(out3.shape)


torch.Size([2, 128, 100])
torch.Size([2, 128, 100])
torch.Size([2, 128, 100])
torch.Size([2, 64, 100])
